In [1]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import yfinance as yf

#Used Models
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
symbol='^NSEI'
ticker = yf.Ticker(symbol)
data=ticker.history(period='max',interval='1m')
data.reset_index(inplace=True)

In [3]:
data.head()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-18 09:15:00+05:30,22224.449219,22250.550781,22215.199219,22249.949219,0,0.0,0.0
1,2024-04-18 09:16:00+05:30,22249.199219,22257.300781,22246.849609,22253.000000,0,0.0,0.0
2,2024-04-18 09:17:00+05:30,22252.400391,22254.550781,22240.699219,22253.400391,0,0.0,0.0
3,2024-04-18 09:18:00+05:30,22253.949219,22257.550781,22250.050781,22257.349609,0,0.0,0.0
4,2024-04-18 09:19:00+05:30,22256.050781,22259.250000,22246.500000,22246.800781,0,0.0,0.0


In [4]:
data.tail()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
1869,2024-04-24 15:25:00+05:30,22409.650391,22413.300781,22409.500000,22409.949219,0,0.0,0.0
1870,2024-04-24 15:26:00+05:30,22409.650391,22414.949219,22408.949219,22410.250000,0,0.0,0.0
1871,2024-04-24 15:27:00+05:30,22411.050781,22412.650391,22408.550781,22409.449219,0,0.0,0.0
1872,2024-04-24 15:28:00+05:30,22409.650391,22410.900391,22404.849609,22405.849609,0,0.0,0.0
1873,2024-04-24 15:29:00+05:30,22407.650391,22414.300781,22405.050781,22412.199219,0,0.0,0.0


In [5]:
# data = pd.read_csv('/content/Nifty50_shortTerm_5m_New.csv')

print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1874 entries, 0 to 1873
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                       
---  ------        --------------  -----                       
 0   Datetime      1874 non-null   datetime64[ns, Asia/Kolkata]
 1   Open          1874 non-null   float64                     
 2   High          1874 non-null   float64                     
 3   Low           1874 non-null   float64                     
 4   Close         1874 non-null   float64                     
 5   Volume        1874 non-null   int64                       
 6   Dividends     1874 non-null   float64                     
 7   Stock Splits  1874 non-null   float64                     
dtypes: datetime64[ns, Asia/Kolkata](1), float64(6), int64(1)
memory usage: 117.2 KB
None


In [6]:
print(data.describe())


               Open          High           Low         Close  Volume  \
count   1874.000000   1874.000000   1874.000000   1874.000000  1874.0   
mean   22260.091645  22264.369054  22255.495139  22260.138191     0.0   
std      175.910187    174.433318    177.296983    175.862972     0.0   
min    21793.750000  21797.500000  21778.550781  21794.400391     0.0   
25%    22152.662598  22160.512695  22145.899902  22152.787598     0.0   
50%    22293.099609  22296.224609  22290.025391  22293.025391     0.0   
75%    22402.150391  22404.437012  22397.538086  22402.000000     0.0   
max    22474.150391  22475.900391  22471.400391  22473.750000     0.0   

       Dividends  Stock Splits  
count     1874.0        1874.0  
mean         0.0           0.0  
std          0.0           0.0  
min          0.0           0.0  
25%          0.0           0.0  
50%          0.0           0.0  
75%          0.0           0.0  
max          0.0           0.0  


In [7]:
data.drop_duplicates()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-18 09:15:00+05:30,22224.449219,22250.550781,22215.199219,22249.949219,0,0.0,0.0
1,2024-04-18 09:16:00+05:30,22249.199219,22257.300781,22246.849609,22253.000000,0,0.0,0.0
2,2024-04-18 09:17:00+05:30,22252.400391,22254.550781,22240.699219,22253.400391,0,0.0,0.0
3,2024-04-18 09:18:00+05:30,22253.949219,22257.550781,22250.050781,22257.349609,0,0.0,0.0
4,2024-04-18 09:19:00+05:30,22256.050781,22259.250000,22246.500000,22246.800781,0,0.0,0.0
...,...,...,...,...,...,...,...,...
1869,2024-04-24 15:25:00+05:30,22409.650391,22413.300781,22409.500000,22409.949219,0,0.0,0.0
1870,2024-04-24 15:26:00+05:30,22409.650391,22414.949219,22408.949219,22410.250000,0,0.0,0.0
1871,2024-04-24 15:27:00+05:30,22411.050781,22412.650391,22408.550781,22409.449219,0,0.0,0.0
1872,2024-04-24 15:28:00+05:30,22409.650391,22410.900391,22404.849609,22405.849609,0,0.0,0.0


In [8]:
data.isnull().sum()

Datetime        0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

In [9]:
data.drop(['Datetime','Volume','Dividends','Stock Splits'],axis=1,inplace=True)
data.head(10)



,Open,High,Low,Close
0,22224.449219,22250.550781,22215.199219,22249.949219
1,22249.199219,22257.300781,22246.849609,22253.000000
2,22252.400391,22254.550781,22240.699219,22253.400391
3,22253.949219,22257.550781,22250.050781,22257.349609
4,22256.050781,22259.250000,22246.500000,22246.800781
5,22246.300781,22253.750000,22244.900391,22251.150391
6,22251.599609,22259.500000,22251.449219,22257.250000
7,22257.300781,22257.300781,22248.099609,22255.599609
8,22254.150391,22256.000000,22252.250000,22255.400391
9,22262.650391,22269.500000,22262.199219,22267.449219


In [10]:
data['shift_close'] = data['Close'].shift(-1)
data.dropna(inplace=True)

In [11]:
data

,Open,High,Low,Close,shift_close
0,22224.449219,22250.550781,22215.199219,22249.949219,22253.000000
1,22249.199219,22257.300781,22246.849609,22253.000000,22253.400391
2,22252.400391,22254.550781,22240.699219,22253.400391,22257.349609
3,22253.949219,22257.550781,22250.050781,22257.349609,22246.800781
4,22256.050781,22259.250000,22246.500000,22246.800781,22251.150391
...,...,...,...,...,...
1868,22408.900391,22410.949219,22407.250000,22409.099609,22409.949219
1869,22409.650391,22413.300781,22409.500000,22409.949219,22410.250000
1870,22409.650391,22414.949219,22408.949219,22410.250000,22409.449219
1871,22411.050781,22412.650391,22408.550781,22409.449219,22405.849609


In [12]:
data.tail()

,Open,High,Low,Close,shift_close
1868,22408.900391,22410.949219,22407.250000,22409.099609,22409.949219
1869,22409.650391,22413.300781,22409.500000,22409.949219,22410.250000
1870,22409.650391,22414.949219,22408.949219,22410.250000,22409.449219
1871,22411.050781,22412.650391,22408.550781,22409.449219,22405.849609
1872,22409.650391,22410.900391,22404.849609,22405.849609,22412.199219


In [13]:
X=data.drop('shift_close',axis=1)
y = data['shift_close']

In [14]:
xy=pd.concat([X,y],axis=1)
xy.tail()

,Open,High,Low,Close,shift_close
1868,22408.900391,22410.949219,22407.250000,22409.099609,22409.949219
1869,22409.650391,22413.300781,22409.500000,22409.949219,22410.250000
1870,22409.650391,22414.949219,22408.949219,22410.250000,22409.449219
1871,22411.050781,22412.650391,22408.550781,22409.449219,22405.849609
1872,22409.650391,22410.900391,22404.849609,22405.849609,22412.199219


In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

predictions = my_model.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(my_model.score(X_test,y_test)))

predictions


Mean Absolute Error: 6.169401041666666
R2 Score0.9971780499402236
Accuracy Score : 0.9971780499402236


array([22464.512, 22126.352, 22390.031, 22200.537, 22393.639, 22411.63 ,
       22399.37 , 22243.084, 22457.287, 22196.4  , 22452.578, 22411.186,
       22448.43 , 22159.25 , 21973.94 , 22450.086, 22410.898, 22411.63 ,
       22389.314, 21984.812, 22250.172, 22289.756, 22390.254, 22409.543,
       22397.521, 22258.973, 22240.322, 22408.723, 22413.021, 21876.342,
       22451.346, 22377.895, 22373.506, 22252.766, 21867.05 , 22344.951,
       22390.373, 22282.379, 22317.906, 22286.574, 22412.688, 22289.633,
       22315.553, 22237.744, 22409.27 , 22293.96 , 22308.873, 22223.64 ,
       22294.895, 22285.31 , 22449.719, 22380.908, 22293.2  , 22453.5  ,
       22035.898, 22447.23 , 22233.332, 22285.848, 22195.484, 22245.295,
       22411.506, 22277.826, 22293.832, 22220.518, 22283.2  , 22208.322,
       22232.445, 22244.672, 22061.045, 22411.688, 22184.914, 22173.545,
       22258.297, 22437.543, 22447.342, 21869.648, 22445.156, 22451.6  ,
       22293.385, 22293.912, 22403.7  , 22316.795, 

In [17]:
RandomeForestModel = RandomForestRegressor()
RandomeForestModel.fit(X_train, y_train)


predictions = RandomeForestModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(RandomeForestModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 5.893632552083233
R2 Score0.9969847753230328
Accuracy Score : 0.9969847753230328


array([22463.41142578, 22126.35554687, 22390.78492188, 22199.56591797,
       22394.42611328, 22411.31066406, 22400.23929687, 22242.23210937,
       22456.00673828, 22189.54689453, 22450.68943359, 22410.27011719,
       22451.52408203, 22160.39628906, 21969.92103516, 22451.11222656,
       22408.76785156, 22411.00001953, 22389.13837891, 21983.06724609,
       22251.02566406, 22289.44152344, 22394.12091797, 22408.76080078,
       22397.67345703, 22257.86535156, 22242.56464844, 22407.96742188,
       22412.89023438, 21874.99208984, 22451.05111328, 22380.04410156,
       22372.32455078, 22255.27132812, 21867.97873047, 22347.98193359,
       22387.42238281, 22279.05796875, 22318.97255859, 22288.46011719,
       22409.62914062, 22289.18730469, 22312.48617187, 22238.02789062,
       22409.73830078, 22294.73322266, 22312.65603516, 22222.22972656,
       22292.57138672, 22289.72970703, 22449.19046875, 22376.87035156,
       22293.26353516, 22454.18259766, 22038.79044922, 22445.755625  ,
      

In [18]:
DesitionTreeModel = DecisionTreeRegressor()
DesitionTreeModel.fit(X_train, y_train)


predictions = DesitionTreeModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(DesitionTreeModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 7.213364583333333
R2 Score0.9950141087656996
Accuracy Score : 0.9950141087656996


array([22461.84960938, 22122.40039062, 22389.25      , 22202.        ,
       22390.94921875, 22412.55078125, 22390.09960938, 22243.09960938,
       22455.84960938, 22196.75      , 22448.65039062, 22412.30078125,
       22453.34960938, 22155.09960938, 21967.05078125, 22452.05078125,
       22409.30078125, 22408.5       , 22388.25      , 21985.80078125,
       22255.19921875, 22290.80078125, 22390.25      , 22412.19921875,
       22396.30078125, 22268.34960938, 22232.5       , 22411.        ,
       22409.44921875, 21876.09960938, 22451.09960938, 22376.09960938,
       22367.40039062, 22246.80078125, 21866.59960938, 22348.65039062,
       22383.80078125, 22278.05078125, 22316.30078125, 22291.69921875,
       22405.40039062, 22288.59960938, 22313.5       , 22246.94921875,
       22405.40039062, 22293.15039062, 22311.25      , 22218.90039062,
       22291.94921875, 22289.40039062, 22447.        , 22370.65039062,
       22294.5       , 22454.05078125, 22044.44921875, 22442.75      ,
      

In [19]:

import pickle
pickle.dump(my_model,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl','wb'))
pickle.dump(RandomeForestModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl','wb'))
pickle.dump(DesitionTreeModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl','wb'))

# pickle.dump(my_model,open('/content/drive/MyDrive/AI COLLAGE/5m_Nifty_RandomForestRegressor.pkl','wb'))


import yfinance as yf
import datetime as dt
import pickle
from sklearn.metrics import mean_absolute_error,r2_score,accuracy


# Load the model from the pickle file
with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl', 'rb') as f:
    Rand = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl', 'rb') as f:
    Decision = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl', 'rb') as f:
    XgBoostModel = pickle.load(f)



while True:
    ticker = yf.Ticker('^NSEI')
    start=dt.datetime.now().strftime('%Y-%m-%d')
    end=(dt.datetime.now() + dt.timedelta(days=1)).strftime('%Y-%m-%d')

    data = ticker.history(interval='5m',start=start,end=end)
    data.reset_index(inplace=True)
    # print(data)
    data.columns = data.columns.str.strip()  # Remove extra spaces from column names

    data.drop(['Datetime', 'Volume', 'Dividends', 'Stock Splits'], inplace=True, axis=1)
    y = data['Close'].copy()
    data.drop(['Close'], inplace=True, axis=1)

    print("Random Forest Regressor")
    print("---------------------------\n")
    Rand_predic=Rand.predict(data)
    last_pred=Rand_predic[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,Rand_predic)}')
    # print(f"Rand Accuracy Score :{}")
    print(f'r2 : {r2_score(y,Rand_predic)}')
    print("---------------------------")
    print("\n\n\n")

    print("Desicion Tree Regressor")
    print("---------------------------\n")
    DecisionPred=Decision.predict(data)
    last_pred=DecisionPred[-1]
    mae=mean_absolute_error(y,DecisionPred)
    # last_pred=last_pred+mae
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mae}')
    print(f'r2 : {r2_score(y,DecisionPred)}')
    print("---------------------------")
    print("\n\n\n")

    print("XgBoostModel ")
    print("---------------------------\n")
    XgBoostPred=XgBoostModel.predict(data)
    last_pred=XgBoostPred[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,XgBoostPred)}')
    print(f'r2 : {r2_score(y,XgBoostPred)}')
    print("---------------------------")
    print("\n\n\n")
    # data